In [15]:
import os
import sys

current_dir = os.getcwd()
# print(current_dir)
project_root = os.path.abspath(os.path.join(current_dir, "..","..", ".."))
# print(project_root)
sys.path.append(project_root)

In [ ]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

In [1]:
from pyspark.sql.functions import lit, col, create_map
from src.citibike.citibike_utils import get_trip_duration_mins
from src.utils.datetime_utils import timestamp_to_date_col

In [ ]:
df = spark.read.table(f"{catalog}.01_bronze.jc_citibike")

In [4]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [5]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [ ]:
df = df.withColumn("metadata",
                   create_map(
                        lit("pipeline_id"), lit(pipeline_id),
                        lit("run_id"), lit(run_id),
                        lit("task_id"), lit(task_id), 
                        lit("processed_timestamp"), lit(processed_timestamp)
                      ))

In [8]:
df = df.select("ride_id", "trip_start_date", "started_at", "ended_at", "start_station_name", 
             "end_station_name", "trip_duration_mins", "metadata")

In [ ]:
df.write.mode('overwrite').option('overwriteSchema', 'true').saveAsTable(f"{catalog}.02_silver.jc_citibike")